In [327]:
import pandas as pd
import ast
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
import json
import numpy as np

# !pip install --upgrade scikit-learn

In [329]:
movie_1 = pd.read_csv("Movies_no_revenue_zero.csv")

In [331]:
movie_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20759 entries, 0 to 20758
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movieId               20759 non-null  float64
 1   imdb_id               20759 non-null  object 
 2   original_title        20759 non-null  object 
 3   production_companies  20759 non-null  object 
 4   production_countries  20759 non-null  object 
 5   release_date          20752 non-null  object 
 6   runtime               20695 non-null  float64
 7   title                 20759 non-null  object 
 8   budget                20759 non-null  float64
 9   revenue               20759 non-null  float64
 10  genres                20759 non-null  object 
 11  popularity            20759 non-null  float64
 12  vote_average          20759 non-null  float64
 13  vote_count            20759 non-null  float64
 14  director              20592 non-null  object 
 15  lead_cast          

In [333]:
movie_1["release_date"] = pd.to_datetime(movie_1["release_date"])
movie_1.head(5).to_string()

"   movieId    imdb_id               original_title                                                                                                                 production_companies                                        production_countries release_date  runtime                        title      budget      revenue                              genres  popularity  vote_average  vote_count         director                              lead_cast\n0    862.0  tt0114709                    Toy Story                                                                                       [{'name': 'Pixar Animation Studios', 'id': 3}]  [{'iso_3166_1': 'US', 'name': 'United States of America'}]   1995-10-30     81.0                    Toy Story  30000000.0  373554033.0   ['Animation', 'Comedy', 'Family']   21.946943           7.7      5415.0    John Lasseter             ['Tom Hanks', 'Tim Allen']\n1   8844.0  tt0113497                      Jumanji  [{'name': 'TriStar Pictures', 'id': 559}, {'n

In [335]:
movie_1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20759 entries, 0 to 20758
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movieId               20759 non-null  float64       
 1   imdb_id               20759 non-null  object        
 2   original_title        20759 non-null  object        
 3   production_companies  20759 non-null  object        
 4   production_countries  20759 non-null  object        
 5   release_date          20752 non-null  datetime64[ns]
 6   runtime               20695 non-null  float64       
 7   title                 20759 non-null  object        
 8   budget                20759 non-null  float64       
 9   revenue               20759 non-null  float64       
 10  genres                20759 non-null  object        
 11  popularity            20759 non-null  float64       
 12  vote_average          20759 non-null  float64       
 13  vote_count      

In [337]:
movie_1["production_companies"] = movie_1["production_companies"].apply(lambda x: [entry['name'] for entry in ast.literal_eval(x)])
print(movie_1[["production_companies"]])

                                    production_companies
0                              [Pixar Animation Studios]
1      [TriStar Pictures, Teitler Film, Interscope Co...
2                         [Warner Bros., Lancaster Gate]
3               [Twentieth Century Fox Film Corporation]
4           [Sandollar Productions, Touchstone Pictures]
...                                                  ...
20754                                                 []
20755  [Art Pictures Studio, Art Pictures Group, Task...
20756                                                 []
20757                    [Mad Films, Third Eye Pictures]
20758                                                 []

[20759 rows x 1 columns]


In [338]:
A = movie_1[movie_1["production_companies"].apply(len) == 0]
print(A.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3755 entries, 54 to 20758
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movieId               3755 non-null   float64       
 1   imdb_id               3755 non-null   object        
 2   original_title        3755 non-null   object        
 3   production_companies  3755 non-null   object        
 4   production_countries  3755 non-null   object        
 5   release_date          3748 non-null   datetime64[ns]
 6   runtime               3716 non-null   float64       
 7   title                 3755 non-null   object        
 8   budget                3755 non-null   float64       
 9   revenue               3755 non-null   float64       
 10  genres                3755 non-null   object        
 11  popularity            3755 non-null   float64       
 12  vote_average          3755 non-null   float64       
 13  vote_count           

In [339]:
movie_1.loc[movie_1["production_companies"].apply(len) == 0, "production_companies"] = "Not Announced"
A = movie_1[movie_1["production_companies"].apply(len) == 0]
print(A.info())
movie_1.to_csv("test-m-0.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movieId               0 non-null      float64       
 1   imdb_id               0 non-null      object        
 2   original_title        0 non-null      object        
 3   production_companies  0 non-null      object        
 4   production_countries  0 non-null      object        
 5   release_date          0 non-null      datetime64[ns]
 6   runtime               0 non-null      float64       
 7   title                 0 non-null      object        
 8   budget                0 non-null      float64       
 9   revenue               0 non-null      float64       
 10  genres                0 non-null      object        
 11  popularity            0 non-null      float64       
 12  vote_average          0 non-null      float64       
 13  vote_count            0 non-null     

In [343]:
movie_1_production_company_exploded = movie_1.explode('production_companies')

# test_df = pd.get_dummies(movie_1_exploded, columns=['production_companies'], prefix='prod_comp', dtype='int')
# # encoder = OneHotEncoder(sparse_output=False, drop='first')
# # encoded_data = encoder.fit_transform(movie_1_exploded)
# # encoded_df = test_df.filter(like='prod_comp')

# encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
# encoded_data = encoder.fit_transform(movie_1_exploded[['production_companies']])
# encoded_dense = encoded_data.toarray()

# # Now create DataFrame with correct number of columns
# encoded_df = pd.DataFrame(encoded_dense, columns=encoder.get_feature_names_out(['production_companies']))

# print(encoded_data)
# test_df.to_csv("test-m-3.csv")
# one_hot_encode = pd.get_dummies(movie_1_exploded['production_companies'])


# encoder = OneHotEncoder(handle_unknown="ignore")
# encoded_features = encoder.fit_transform(movie_1[["production_companies"]])

# encoded_movie_production_companies = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out("production_companies"))
# print(encoded_movie_production_companies.head(5))


# result = one_hot_encode.max().to_list()
# print("One-Hot Encoded (as list):", result)
# print("\nOne-Hot Encoded (DataFrame - intermediate):")

# print(one_hot_encode.head(5))

# one_hot_encode.to_csv("movie-prod.csv", index=False)

In [345]:
production_company_freq_encoding = movie_1_production_company_exploded['production_companies'].value_counts(normalize=True)
movie_1_production_company_exploded['prod_comp_encoded'] = movie_1_production_company_exploded['production_companies'].map(production_company_freq_encoding)

In [347]:
movie_1_production_company_encoded = movie_1_production_company_exploded.groupby("movieId").agg({'prod_comp_encoded': 'mean'}).reset_index()

In [349]:
movie_1_with_production_company = movie_1.drop(columns=['production_companies']).merge(movie_1_production_company_encoded, on="movieId")
movie_1_with_production_company.to_csv("Prod-Comp.csv", index=False)

In [351]:
# ipca = IncrementalPCA(n_components=500) 
# for batch in range(0, encoded_df.shape[0], 1000):  # Process in batches of 1000
#     ipca.partial_fit(encoded_df[batch:batch+1000].toarray())

# pca_transformed = ipca.transform(encoded_df.toarray())

# ipca = IncrementalPCA(n_components=500)
# for batch in range(0, encoded_df.shape[0], 1000):
#     ipca.partial_fit(encoded_df[batch:batch+1000].to_numpy()) # Use .to_numpy()

# pca_transformed = ipca.transform(encoded_df.to_numpy())
# print(pca_transformed)

# pca = PCA(n_components=0.80)
# pca_result = pca.fit_transform(encoded_df)
# print(pca_result)
# # Convert PCA features to DataFrame
# pca_columns = [f"PCA_Comp_{i+1}" for i in range(pca_transformed.shape[1])]
# pca_df = pd.DataFrame(pca_transformed, columns=pca_columns)

In [353]:
movie_1_with_production_company["production_countries"] = movie_1_with_production_company["production_countries"].apply(lambda x: [entry['name'] for entry in ast.literal_eval(x)])
print(movie_1_with_production_company[["production_countries"]])

             production_countries
0      [United States of America]
1      [United States of America]
2      [United States of America]
3      [United States of America]
4      [United States of America]
...                           ...
20754                    [Russia]
20755                    [Russia]
20756                          []
20757                     [India]
20758            [United Kingdom]

[20759 rows x 1 columns]


In [354]:
A = movie_1_with_production_company[movie_1_with_production_company["production_countries"].apply(len) == 0]
print(A.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1593 entries, 79 to 20756
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movieId               1593 non-null   float64       
 1   imdb_id               1593 non-null   object        
 2   original_title        1593 non-null   object        
 3   production_countries  1593 non-null   object        
 4   release_date          1586 non-null   datetime64[ns]
 5   runtime               1571 non-null   float64       
 6   title                 1593 non-null   object        
 7   budget                1593 non-null   float64       
 8   revenue               1593 non-null   float64       
 9   genres                1593 non-null   object        
 10  popularity            1593 non-null   float64       
 11  vote_average          1593 non-null   float64       
 12  vote_count            1593 non-null   float64       
 13  director             

In [357]:
movie_1_with_production_company.loc[movie_1_with_production_company["production_countries"].apply(len) == 0, "production_countries"] = "Not Announced"
A = movie_1_with_production_company[movie_1_with_production_company["production_countries"].apply(len) == 0]
print(A.info())
movie_1_with_production_company.to_csv("test-m-6.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   movieId               0 non-null      float64       
 1   imdb_id               0 non-null      object        
 2   original_title        0 non-null      object        
 3   production_countries  0 non-null      object        
 4   release_date          0 non-null      datetime64[ns]
 5   runtime               0 non-null      float64       
 6   title                 0 non-null      object        
 7   budget                0 non-null      float64       
 8   revenue               0 non-null      float64       
 9   genres                0 non-null      object        
 10  popularity            0 non-null      float64       
 11  vote_average          0 non-null      float64       
 12  vote_count            0 non-null      float64       
 13  director              0 non-null     

In [359]:
movie_1_production_country_exploded = movie_1_with_production_company.explode('production_countries')

movie_1_prod_country = pd.get_dummies(movie_1_production_country_exploded, columns=['production_countries'], prefix='prod_country', dtype='int')
# # encoder = OneHotEncoder(sparse_output=False, drop='first')
# # encoded_data = encoder.fit_transform(movie_1_exploded)
encoded_df = movie_1_prod_country.filter(like='prod_country')

# encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
# encoded_data = encoder.fit_transform(movie_1_exploded[['production_companies']])
# encoded_dense = encoded_data.toarray()

# # Now create DataFrame with correct number of columns
# encoded_df = pd.DataFrame(encoded_dense, columns=encoder.get_feature_names_out(['production_companies']))

# print(encoded_data)
movie_1_prod_country.to_csv("test-m-7.csv")
# one_hot_encode = pd.get_dummies(movie_1_exploded['production_companies'])


# encoder = OneHotEncoder(handle_unknown="ignore")
# encoded_features = encoder.fit_transform(movie_1[["production_companies"]])

# encoded_movie_production_companies = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out("production_companies"))
# print(encoded_movie_production_companies.head(5))


In [360]:
pca = PCA(n_components=0.80)
pca_result = pca.fit_transform(encoded_df)
print(pca_result)

[[ 6.53932872e-01  1.26719254e-02  2.12671203e-02 ...  1.26141592e-03
   4.27367393e-04  1.06759664e-04]
 [ 6.53932872e-01  1.26719254e-02  2.12671203e-02 ...  1.26141592e-03
   4.27367393e-04  1.06759664e-04]
 [ 6.53932872e-01  1.26719254e-02  2.12671203e-02 ...  1.26141592e-03
   4.27367393e-04  1.06759664e-04]
 ...
 [-4.00523612e-01 -1.27731978e-01 -5.19283412e-01 ...  1.00165098e-02
   3.31867349e-03  8.22659669e-04]
 [-3.34375647e-01 -5.41392312e-02 -1.26092554e-01 ...  6.74549857e-02
   1.95218177e-02  4.63943611e-03]
 [-4.51598025e-01 -5.73117518e-01  6.63433297e-01 ...  6.73691888e-03
   2.25069052e-03  5.59507507e-04]]


In [361]:
pca_df = pd.DataFrame(pca_result)
movie_1_with_production_company = movie_1_with_production_company.reset_index(drop=True)
pca_df = pca_df.reset_index(drop=True)
movie_1_with_production_company = movie_1_with_production_company.drop(columns=['production_countries'])
movie_1_with_production_company.info()
movie_1_with_production_countries = pd.concat([movie_1_with_production_company, pca_df], axis=1)
movie_1_with_production_countries.to_csv('test-m-8.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20759 entries, 0 to 20758
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            20759 non-null  float64       
 1   imdb_id            20759 non-null  object        
 2   original_title     20759 non-null  object        
 3   release_date       20752 non-null  datetime64[ns]
 4   runtime            20695 non-null  float64       
 5   title              20759 non-null  object        
 6   budget             20759 non-null  float64       
 7   revenue            20759 non-null  float64       
 8   genres             20759 non-null  object        
 9   popularity         20759 non-null  float64       
 10  vote_average       20759 non-null  float64       
 11  vote_count         20759 non-null  float64       
 12  director           20592 non-null  object        
 13  lead_cast          20759 non-null  object        
 14  prod_c

In [363]:
# A = movie_1_with_production_countries[movie_1_with_production_countries["genres"].apply(len) == 0]
# print(A.info())

In [366]:
movie_1_with_production_countries["genres"] = (movie_1_with_production_countries["genres"].dropna().map(ast.literal_eval))

# Use explode() safely
movie_1_genre_exploded = movie_1_with_production_countries.explode("genres", ignore_index=True)

# movie_1_with_production_countries["genres"] = movie_1_with_production_countries["genres"].apply(ast.literal_eval)

# movie_1_genre_exploded = movie_1_with_production_countries.explode('genres')
# movie_1_genre = pd.get_dummies(movie_1_genre_exploded, columns=['genres'], prefix='genre_', dtype='int')

# encoded_df = movie_1_genre.filter(like='prod_country')

movie_1_genre_exploded.to_csv("test-m-9.csv")

In [367]:
genre_freq_encoding = movie_1_genre_exploded['genres'].value_counts(normalize=True)
movie_1_genre_exploded['genre_encoded'] = movie_1_genre_exploded['genres'].map(genre_freq_encoding)

In [368]:
movie_1_genre_encoded = movie_1_genre_exploded.groupby("movieId").agg({'genre_encoded': 'mean'}).reset_index()

In [369]:
movie_1_with_genre = movie_1_with_production_countries.drop(columns=['genres']).merge(movie_1_genre_encoded, on="movieId")
movie_1_with_genre.to_csv("Genre.csv", index=False)

In [373]:
# movie_1_with_production_countries["genres"] = (movie_1_with_production_countries["genres"].dropna().map(ast.literal_eval))

# # Use explode() safely
# movie_1_genre_exploded = movie_1_with_production_countries.explode("genres", ignore_index=True)

movie_1_with_genre["lead_cast"] = (movie_1_with_genre["lead_cast"].dropna().map(ast.literal_eval))

movie_1_actors_exploded = movie_1_with_genre.explode('lead_cast', ignore_index=True)
# movie_1_actors = pd.get_dummies(movie_1_actors_exploded, columns=['lead_cast'], prefix='actor_', dtype='int')

# encoded_df = movie_1_genre.filter(like='prod_country')

# movie_1_actors_exploded.to_csv("test-m-10.csv")

In [374]:
actor_freq_encoding = movie_1_actors_exploded['lead_cast'].value_counts(normalize=True)
movie_1_actors_exploded['actor_encoded'] = movie_1_actors_exploded['lead_cast'].map(actor_freq_encoding)

In [375]:
movie_1_actor_encoded = movie_1_actors_exploded.groupby("movieId").agg({'actor_encoded': 'mean'}).reset_index()

In [376]:
movie_1_with_actor = movie_1_with_genre.drop(columns=['lead_cast']).merge(movie_1_actor_encoded, on="movieId")
movie_1_with_actor.to_csv("Actors.csv", index=False)

In [378]:
movie_1_with_actor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20759 entries, 0 to 20758
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            20759 non-null  float64       
 1   imdb_id            20759 non-null  object        
 2   original_title     20759 non-null  object        
 3   release_date       20752 non-null  datetime64[ns]
 4   runtime            20695 non-null  float64       
 5   title              20759 non-null  object        
 6   budget             20759 non-null  float64       
 7   revenue            20759 non-null  float64       
 8   popularity         20759 non-null  float64       
 9   vote_average       20759 non-null  float64       
 10  vote_count         20759 non-null  float64       
 11  director           20592 non-null  object        
 12  prod_comp_encoded  20759 non-null  float64       
 13  0                  20759 non-null  float64       
 14  1     

In [390]:
movie_1_with_actor['genre_encoded'] = movie_1_with_actor['genre_encoded'].fillna(movie_1_with_actor['genre_encoded'].mean())

# movie_1_with_actor['genre_encoded'].fillna(movie_1_with_actor['genre_encoded'].mean(), inplace = True)

movie_1_with_actor.dropna(subset=["actor_encoded"], inplace=True)
movie_1_with_actor["runtime"] = movie_1_with_actor["runtime"].replace("", np.nan)
movie_1_with_actor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20232 entries, 0 to 20757
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            20232 non-null  float64       
 1   imdb_id            20232 non-null  object        
 2   original_title     20232 non-null  object        
 3   release_date       20230 non-null  datetime64[ns]
 4   runtime            20175 non-null  float64       
 5   title              20232 non-null  object        
 6   budget             20232 non-null  float64       
 7   revenue            20232 non-null  float64       
 8   popularity         20232 non-null  float64       
 9   vote_average       20232 non-null  float64       
 10  vote_count         20232 non-null  float64       
 11  director           20142 non-null  object        
 12  prod_comp_encoded  20232 non-null  float64       
 13  0                  20232 non-null  float64       
 14  1          

In [394]:
movie_1_with_actor.dropna(subset=["release_date"], inplace=True)

# movie_1_with_actor.dropna(subset=[""], inplace=True)
movie_1_with_actor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20230 entries, 0 to 20757
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            20230 non-null  float64       
 1   imdb_id            20230 non-null  object        
 2   original_title     20230 non-null  object        
 3   release_date       20230 non-null  datetime64[ns]
 4   runtime            20173 non-null  float64       
 5   title              20230 non-null  object        
 6   budget             20230 non-null  float64       
 7   revenue            20230 non-null  float64       
 8   popularity         20230 non-null  float64       
 9   vote_average       20230 non-null  float64       
 10  vote_count         20230 non-null  float64       
 11  director           20141 non-null  object        
 12  prod_comp_encoded  20230 non-null  float64       
 13  0                  20230 non-null  float64       
 14  1          

In [398]:
movie_1_with_actor["runtime"] = movie_1_with_actor["runtime"].replace("", "-")

movie_1_with_actor["director"] = movie_1_with_actor["director"].replace("", "-")

movie_1_with_actor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20230 entries, 0 to 20757
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movieId            20230 non-null  float64       
 1   imdb_id            20230 non-null  object        
 2   original_title     20230 non-null  object        
 3   release_date       20230 non-null  datetime64[ns]
 4   runtime            20173 non-null  float64       
 5   title              20230 non-null  object        
 6   budget             20230 non-null  float64       
 7   revenue            20230 non-null  float64       
 8   popularity         20230 non-null  float64       
 9   vote_average       20230 non-null  float64       
 10  vote_count         20230 non-null  float64       
 11  director           20141 non-null  object        
 12  prod_comp_encoded  20230 non-null  float64       
 13  0                  20230 non-null  float64       
 14  1          

In [400]:
final_df = movie_1_with_actor.to_csv("final_movie_dataset.csv", index=False)